In [6]:
import mmcv
from mmdet.apis import init_detector, inference_detector
model = init_detector(config="balloon.py", checkpoint="work_dirs/balloon/latest.pth")
v = mmcv.VideoReader("balloon_color_splash.gif")
v.cvt2frames("frame_dir/in")

load checkpoint from local path: work_dirs/balloon/latest.pth
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 78/78, 60.7 task/s, elapsed: 1s, ETA:     0s


In [7]:
import numpy as np
import os

def pre_single_pic(img):
    result = inference_detector(model, img)
    bbox, segm = result
    bbox = np.array(bbox[0]) # bbox 为列表
#     print(bbox.shape) # (23, 5)
    segm = np.array(segm[0])
#     print(segm)
#     print(segm.shape) # (23, 1024, 683)
    score_thr = 0.75
    balloon_cnt = bbox[bbox[:, -1] > score_thr].shape[0]
    data = mmcv.imread(img)  # -> ndarray
    tmp = data.copy()
    # Gray = (Red * 0.3 + Green * 0.59 + Blue * 0.11)
    # 'bgr'
    data[:, :, 0] = data[:, :, 2] * 0.3 + data[:, :, 1] * 0.59 + data[:, :, 0] * 0.11
    data[:, :, 1] = data[:, :, 0]
    data[:, :, 2] = data[:, :, 0]
    for balloon in segm[:balloon_cnt]:
        data[balloon] = tmp[balloon]
    return data

In [8]:
file_path = "frame_dir/in/"
result_path = "frame_dir/result/"
if not os.path.exists(result_path):
    os.makedirs(result_path)

In [9]:
for root, dirs, files in os.walk(file_path, topdown=False):
    for image_id in files:
        filename = file_path + image_id
        savename = result_path+image_id
        data = pre_single_pic(filename) 
        mmcv.imwrite(data, savename)

In [10]:
mmcv.frames2video(frame_dir="frame_dir/result", video_file="outputnew.mp4")

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 78/78, 91.9 task/s, elapsed: 1s, ETA:     0s
